In [13]:
!python -m pip install spacy==3.3.3

  Using cached spacy-3.3.3-cp38-cp38-manylinux_2_17_aarch64.manylinux2014_aarch64.whl.metadata (23 kB)
  Using cached thinc-8.0.17-cp38-cp38-manylinux_2_17_aarch64.manylinux2014_aarch64.whl (2.3 MB)
  Using cached pydantic-1.8.2-py3-none-any.whl (126 kB)
  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
Using cached spacy-3.3.3-cp38-cp38-manylinux_2_17_aarch64.manylinux2014_aarch64.whl (6.1 MB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.9.2
    Uninstalling pydantic-1.9.2:
      Successfully uninstalled pydantic-1.9.2
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.12
    Uninstalling thinc-8.1.12:
      Successfully uninstalled thinc-8.1.12
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.0
    Un

In [14]:
!python -m pip install pydantic==2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 933.4 kB/s eta 0:00:00a 0:00:01
  Using cached typing_extensions-4.9.0-py3-none-any.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.6/355.6 kB 3.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 5.2 MB/s eta 0:00:0000:0100:01m
Using cached typing_extensions-4.9.0-py3-none-any.whl (32 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.16.2
    Uninstalling pydantic_core-2.16.2:
      Successfully uninstalled pydantic_core-2.16.2
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.8.2
    Uninstalling pydantic-1.8.2:
      Successfully uninstalled pydantic-1.8.2
ERROR: pip's dependency resolver does not currently take in

In [8]:
import numpy as np
import pandas as pd
import os
import tqdm
import pickle
from tqdm.notebook import tqdm

from functools import lru_cache

from snorkel.labeling import labeling_function
from snorkel.labeling import LabelingFunction
from snorkel.labeling.model import LabelModel
from snorkel.labeling import PandasLFApplier

import glob

from labeling_functions import get_all_lfs

ImportError: cannot import name 'ModelMetaclass' from 'pydantic.main' (/usr/local/lib/python3.8/site-packages/pydantic/main.py)

In [2]:
ABSTAIN = -1
SIMPLE = 0
NOT_SIMPLE = 1
LOST_MEANING = 2

label_map = {-1: "ABSTAIN", 0: "SIMPLE", 1: "NOT_SIMPLE", 2: "LOST_MEANING"}

In [6]:
#current_type = "simp"
current_type = "src"

In [7]:
all_lfs = get_all_lfs()

NameError: name 'get_all_lfs' is not defined

In [6]:
txt = [ '"Sad Eyed Lady of the Lowlands" is a song by the American singer-songwriter Bob Dylan.', 
        "First released as the final track on Dylan's seventh studio album Blonde on Blonde (1966), the song lasts 11 minutes and 23 seconds, and occupies the entire fourth side of the double album. The song was written by Dylan and produced by Bob Johnston. The recording session began at 6 pm on February 15, 1966, at Columbia Studio A, Nashville, Tennessee, but Dylan worked on the lyrics for several hours while the experienced Nashville session musicians hired to accompany him stood by. Four takes were recorded in the early hours of February 16; the final recording was released on Blonde on Blonde. The music is a waltz in 6/8 time.",
        "Some writers have concluded that the song refers to Joan Baez, although most agree that it was composed for Dylan's wife Sara Lownds.",
        "Dylan's lyrics polarized critics. On its release, several reviewers found them impenetrable, but rated the song favorably. Later writers often agree with this, praising the sound, dismissing the lyrics, and rating the song as amongst Dylan's best work."]



for t in txt:
        applier = PandasLFApplier(all_lfs)
        labels = applier.apply(txt, progress_bar=True)

"Sad Eyed Lady of the Lowlands" is a song by the American singer-songwriter Bob Dylan.
First released as the final track on Dylan's seventh studio album Blonde on Blonde (1966), the song lasts 11 minutes and 23 seconds, and occupies the entire fourth side of the double album. The song was written by Dylan and produced by Bob Johnston. The recording session began at 6 pm on February 15, 1966, at Columbia Studio A, Nashville, Tennessee, but Dylan worked on the lyrics for several hours while the experienced Nashville session musicians hired to accompany him stood by. Four takes were recorded in the early hours of February 16; the final recording was released on Blonde on Blonde. The music is a waltz in 6/8 time.
Some writers have concluded that the song refers to Joan Baez, although most agree that it was composed for Dylan's wife Sara Lownds.
Dylan's lyrics polarized critics. On its release, several reviewers found them impenetrable, but rated the song favorably. Later writers often agre

In [3]:
label_path = f"/workspace/vectors/"

In [4]:
if not os.path.isdir(label_path):
    os.mkdir(label_path)

In [19]:
def get_finished_batches(path = f"/workspace/vectors/*"):
    label_paths = glob.glob(path)
    fin_batches = [int(path.split("_")[-1]) for path in label_paths if "labels_" in path and current_type in path]
    return set(fin_batches)

In [20]:
def save_used_lfs(all_lfs, path = f"/workspace/datasets/{selected_dataset}labels/used_lfs.pkl"):
    names = [lf.name for lf in all_lfs]
    pickle.dump(names, open(path, "wb"))

In [21]:
batch_size = 20
start = 0

save_used_lfs(all_lfs)

for i in tqdm(range(start, len(dataset), batch_size), position=1):
    if not i in get_finished_batches():
        try:
            applier = PandasLFApplier(all_lfs)
            labels = applier.apply(dataset[i:i+batch_size], progress_bar=True)
            
            pickle.dump(labels, open(f"/workspace/datasets/{selected_dataset}labels/labels_{current_type}_{i}", "wb"))
            print(f"finished on {i}/{len(dataset)}")

        except Exception as e:
            print(f"something went wrong with batch {i}")
            print(e)

  0%|          | 0/12 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:11<00:00,  1.77it/s]


finished on 0/221


100%|██████████| 20/20 [00:11<00:00,  1.80it/s]


finished on 20/221


...

100%|██████████| 20/20 [00:11<00:00,  1.76it/s]


finished on 60/221


100%|██████████| 20/20 [00:11<00:00,  1.78it/s]


finished on 80/221


 75%|███████▌  | 15/20 [00:08<00:03,  1.63it/s]

In [16]:
#build ds labels
ds_label_path = f"/workspace/datasets/ds_labels"
if not os.path.isdir(ds_label_path):
    os.mkdir(ds_label_path)

In [17]:
def build_labels(ds="MTurkSF"):
    simp_paths = sorted(glob.glob(f"workspace/datasets/{ds}labels/labels_simp*"), key=lambda x: int(x.split("_")[-1]))
    src_paths = sorted(glob.glob(f"workspace/datasets/{ds}labels/labels_src*"), key=lambda x: int(x.split("_")[-1]))

    simp_labels = [pickle.load(open(path, "rb")) for path in simp_paths]
    src_labels = [pickle.load(open(path, "rb")) for path in src_paths]

    simp_labels = np.concatenate(simp_labels)
    src_labels = np.concatenate(src_labels)

    pickle.dump(simp_labels, open(f"{ds_label_path}/{ds}_simp_labels.pkl", "wb"))
    pickle.dump(src_labels, open(f"{ds_label_path}/{ds}_src_labels.pkl", "wb"))

In [ ]:
selected_dataset


In [ ]:
build_labels(selected_dataset)